<a href="https://colab.research.google.com/github/ss-ghule/kaggle_predicting_molecular_properties/blob/master/c4_w2_feature_eng_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Aim: To create final training and test dataframes with new & important features
# Download data

In [0]:
# The code was removed by Watson Studio for sharing.

In [0]:
#!rm train_etl.csv test_etl.csv pairs_dict.pkl connectivity_model_nn_03.h5
!ls

champs-scalar-coupling_new.zip	    scalar_coupling_contributions.csv.zip
dipole_moments.csv.zip		    structures.csv.zip
kaggle.json			    structures.zip
magnetic_shielding_tensors.csv.zip  test.csv.zip
mulliken_charges.csv.zip	    test_etl.csv
potential_energy.csv.zip	    train.csv.zip
sample_data			    train_etl.csv
sample_submission.csv.zip


In [1]:
#download_file_cos(credentials_2, 'train_etl.csv', 'train_etl.csv')
#download_file_cos(credentials_2, 'test_etl.csv', 'test_etl.csv')
#download_file_cos(credentials_2, 'pairs_dict.pkl', 'pairs_dict.pkl')
#download_file_cos(credentials_2, 'connectivity_model_nn_03.h5', 'connectivity_model_nn_03.h5')
!ls

dipole_moments.csv.zip		    scalar_coupling_contributions.csv.zip
kaggle.json			    structures.csv.zip
magnetic_shielding_tensors.csv.zip  structures.zip
mulliken_charges.csv.zip	    test.csv.zip
potential_energy.csv.zip	    test_etl.csv
sample_data			    train.csv.zip
sample_submission.csv.zip	    train_etl.csv


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
from tensorflow import keras

%matplotlib inline

In [3]:
tf.__version__

'1.14.0'

In [4]:
train=pd.read_csv('train_etl.csv')
test=pd.read_csv('test_etl.csv')
print('train:{}'.format(train.shape))
print('test: {}'.format(test.shape))

train:(4658147, 16)
test: (2505542, 15)


In [5]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074


In [6]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,2JHC
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,1JHC
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN,3JHH
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,1JHC
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,2JHC


In [7]:
train.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,4.65815e+06,NaN,NaN,NaN,2.32907e+06,1.34469e+06,0,1.16454e+06,2.32907e+06,3.49361e+06,4.65815e+06
molecule_name,4658147,85003,dsgdb9nsd_042139,135,NaN,NaN,NaN,NaN,NaN,NaN,NaN
atom_index_0,4.65815e+06,NaN,NaN,NaN,13.3569,3.26771,0,11,13,16,28
atom_index_1,4.65815e+06,NaN,NaN,NaN,5.88397,4.99394,0,2,5,8,28
atom_0,4658147,1,H,4658147,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x_0,4.65815e+06,NaN,NaN,NaN,0.105878,1.75987,-9.23489,-0.953092,0.134121,1.22775,9.38224
y_0,4.65815e+06,NaN,NaN,NaN,-0.220369,2.12179,-9.49416,-1.83697,-0.297994,1.75342,9.71447
z_0,4.65815e+06,NaN,NaN,NaN,0.0471289,1.56451,-9.13476,-0.964213,0.00497289,1.0351,7.63758
atom_1,4658147,3,C,3360469,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x_1,4.65815e+06,NaN,NaN,NaN,0.0975718,1.47942,-9.23489,-0.79276,0.0466042,0.968224,8.42089


In [8]:
test.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,2.50554e+06,NaN,NaN,NaN,5.91092e+06,723288,4.65815e+06,5.28453e+06,5.91092e+06,6.5373e+06,7.16369e+06
molecule_name,2505542,45772,dsgdb9nsd_096326,131,NaN,NaN,NaN,NaN,NaN,NaN,NaN
atom_index_0,2.50554e+06,NaN,NaN,NaN,13.3545,3.27221,1,11,13,16,28
atom_index_1,2.50554e+06,NaN,NaN,NaN,5.87795,4.99231,0,2,5,8,28
atom_0,2505542,1,H,2505542,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x_0,2.50554e+06,NaN,NaN,NaN,0.111877,1.75132,-8.35577,-0.946371,0.139294,1.22997,9.15701
y_0,2.50554e+06,NaN,NaN,NaN,-0.223317,2.1128,-9.93394,-1.83265,-0.300268,1.74771,10.182
z_0,2.50554e+06,NaN,NaN,NaN,0.0532973,1.56707,-7.82342,-0.960673,0.00780067,1.04035,7.89473
atom_1,2505542,3,C,1805746,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x_1,2.50554e+06,NaN,NaN,NaN,0.102163,1.4705,-8.35577,-0.775903,0.0472481,0.968822,8.18919


# Feature Engineering (train)

## Euclidian Distance

In [0]:
def get_distance(df):
    return np.sqrt(np.square(df['x_0']-df['x_1'])+np.square(df['y_0']-df['y_1'])+np.square(df['z_0']-df['z_1']))

In [10]:
train['distance']=train.apply(get_distance,axis=1)
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952


## one-hot  encode atom names
(Not neccessary)

In [0]:
#train=train.join(pd.get_dummies(train['atom_0'],prefix='atom_0'))
#train.head()

In [0]:
#train=train.join(pd.get_dummies(train['atom_1'],prefix='atom_1'))
#train.head()

## Label Encode type

In [0]:
from sklearn.preprocessing import LabelEncoder
le_type=LabelEncoder()
train['type_enc']=le_type.fit_transform(train['type'])

In [12]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0


## Label Encode Atoms

In [0]:
le_atoms=LabelEncoder()
train['atom_1_enc']=le_atoms.fit_transform(train['atom_1'])
train['atom_0_enc']=le_atoms.transform(train['atom_0'])

In [14]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,atom_1_enc,atom_0_enc
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0,0,1
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3,1,1
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3,1,1
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3,1,1
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0,0,1


## Pairs

In [0]:
train['pairs']=(train['atom_0']+'_'+train['atom_1']).apply(lambda x:x.split('_')).apply(sorted).apply(lambda x:'_'.join(x)).apply(str.lower)

In [16]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,atom_1_enc,atom_0_enc,pairs
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0,0,1,c_h
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3,1,1,h_h
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3,1,1,h_h
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3,1,1,h_h
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0,0,1,c_h


## Label Encode Pairs using dictionary

In [17]:
!ls

connectivity_model_nn_03.h5	    sample_submission.csv.zip
dipole_moments.csv.zip		    scalar_coupling_contributions.csv.zip
kaggle.json			    structures.csv.zip
magnetic_shielding_tensors.csv.zip  structures.zip
mulliken_charges.csv.zip	    test.csv.zip
pairs_dict.pkl			    test_etl.csv
potential_energy.csv.zip	    train.csv.zip
sample_data			    train_etl.csv


In [0]:
with open('pairs_dict.pkl','rb') as f:
    pairs_dict=pickle.load(f)

In [19]:
pairs_dict

{'c_c': 0,
 'c_f': 1,
 'c_h': 2,
 'c_n': 3,
 'c_o': 4,
 'c_si': 5,
 'f_f': 6,
 'f_h': 7,
 'f_n': 8,
 'f_o': 9,
 'f_si': 10,
 'h_h': 11,
 'h_n': 12,
 'h_o': 13,
 'h_si': 14,
 'n_n': 15,
 'n_o': 16,
 'o_o': 17}

In [20]:
train['pairs'].unique()

array(['c_h', 'h_h', 'h_n'], dtype=object)

In [0]:
train['pairs_enc']=train['pairs'].map(pairs_dict)

In [22]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,atom_1_enc,atom_0_enc,pairs,pairs_enc
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0,0,1,c_h,2
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3,1,1,h_h,11
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3,1,1,h_h,11
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3,1,1,h_h,11
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0,0,1,c_h,2


## Connectivity

In [23]:
model_nn=tf.keras.models.load_model('connectivity_model_nn_03.h5',custom_objects=None,compile=True)

W0816 09:50:57.564599 140606446204800 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0816 09:50:57.566002 140606446204800 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0816 09:50:57.572137 140606446204800 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtyp

In [0]:
x=train[['pairs_enc','distance']]

In [25]:
x.head()

,pairs_enc,distance
0,2,1.091953
1,11,1.783120
2,11,1.783147
3,11,1.783157
4,2,1.091952


In [26]:
x.shape

(4658147, 2)

In [0]:
#model_nn.predict(x,verbose=1)

In [28]:
train['bond']=model_nn.predict(x,verbose=1,batch_size=10000)

4658147/4658147 [==============================] - 4s 1us/sample


In [29]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,atom_1_enc,atom_0_enc,pairs,pairs_enc,bond
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0,0,1,c_h,2,0.919759
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3,1,1,h_h,11,0.648310
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3,1,1,h_h,11,0.648205
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3,1,1,h_h,11,0.648170
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0,0,1,c_h,2,0.919760


In [30]:
train['bond'].isna().value_counts()

False    4658147
Name: bond, dtype: int64

In [0]:
train['bond_int']=train['bond'].apply(lambda x:int(x>0.5))

In [32]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,atom_1_enc,atom_0_enc,pairs,pairs_enc,bond,bond_int
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0,0,1,c_h,2,0.919759,1
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3,1,1,h_h,11,0.648310,1
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3,1,1,h_h,11,0.648205,1
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3,1,1,h_h,11,0.648170,1
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0,0,1,c_h,2,0.919760,1


## Total Charge

In [0]:
train['total_charge']=train['mulliken_charge_0']+train['mulliken_charge_1']

In [0]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,...,-0.535689,1JHC,84.8076,1.091953,0,c_h,2,0.919760,1,-0.401768
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,...,0.133922,2JHH,-11.2570,1.783120,3,h_h,11,0.648310,1,0.267843
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,...,0.133923,2JHH,-11.2548,1.783147,3,h_h,11,0.648205,1,0.267844
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,...,0.133923,2JHH,-11.2543,1.783157,3,h_h,11,0.648170,1,0.267844
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,...,-0.535689,1JHC,84.8074,1.091952,0,c_h,2,0.919760,1,-0.401767


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,...,-0.535689,1JHC,84.8076,1.091953,0,c_h,2,0.919760,1,-0.401768
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,...,0.133922,2JHH,-11.2570,1.783120,3,h_h,11,0.648310,1,0.267843
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,...,0.133923,2JHH,-11.2548,1.783147,3,h_h,11,0.648205,1,0.267844
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,...,0.133923,2JHH,-11.2543,1.783157,3,h_h,11,0.648170,1,0.267844
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,...,-0.535689,1JHC,84.8074,1.091952,0,c_h,2,0.919760,1,-0.401767


## Dipole moment

sum(qi*ri)

In [0]:
train['dp']=(-1*train['mulliken_charge_0']*train['distance']/2)+(train['mulliken_charge_1']*train['distance']/2)

In [0]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge,dp
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,...,1JHC,84.8076,1.091953,0,c_h,2,0.919760,1,-0.401768,-3.655913e-01
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,...,2JHH,-11.2570,1.783120,3,h_h,11,0.648310,1,0.267843,8.915599e-07
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,...,2JHH,-11.2548,1.783147,3,h_h,11,0.648205,1,0.267844,1.783147e-06
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,...,2JHH,-11.2543,1.783157,3,h_h,11,0.648170,1,0.267844,1.783157e-06
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,...,1JHC,84.8074,1.091952,0,c_h,2,0.919760,1,-0.401767,-3.655914e-01


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge,dp
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,...,1JHC,84.8076,1.091953,0,c_h,2,0.919760,1,-0.401768,-3.655913e-01
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,...,2JHH,-11.2570,1.783120,3,h_h,11,0.648310,1,0.267843,8.915599e-07
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,...,2JHH,-11.2548,1.783147,3,h_h,11,0.648205,1,0.267844,1.783147e-06
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,...,2JHH,-11.2543,1.783157,3,h_h,11,0.648170,1,0.267844,1.783157e-06
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,...,1JHC,84.8074,1.091952,0,c_h,2,0.919760,1,-0.401767,-3.655914e-01


## Bond angle, dihedral angle

## Features from External Dataset

In [113]:
#QM9
!wget https://www.kaggleusercontent.com/kf/15612312/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..PMs23Y4b2C47ieojSMw8tg.PXX2plmGLfsUSTlOu4zI97AtOhs7sxSgvZbilx03TjoD6-6XRIfo7cLHEY77C3u7NSxPvqkL9kVii9G55j3q9BaX2sAklQfZJl-VMTEeI0RsRGQlzA9Ve2md5E8RFXqe_csB6wc9u_635F3plGwWPg.g8FiL4J-cls5ky4wsujcWA/data.covs.pickle

--2019-08-16 10:52:26--  https://www.kaggleusercontent.com/kf/15612312/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..PMs23Y4b2C47ieojSMw8tg.PXX2plmGLfsUSTlOu4zI97AtOhs7sxSgvZbilx03TjoD6-6XRIfo7cLHEY77C3u7NSxPvqkL9kVii9G55j3q9BaX2sAklQfZJl-VMTEeI0RsRGQlzA9Ve2md5E8RFXqe_csB6wc9u_635F3plGwWPg.g8FiL4J-cls5ky4wsujcWA/data.covs.pickle
Resolving www.kaggleusercontent.com (www.kaggleusercontent.com)... 35.190.26.106
Connecting to www.kaggleusercontent.com (www.kaggleusercontent.com)|35.190.26.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1678972335 (1.6G) [application/octet-stream]
Saving to: ‘data.covs.pickle’

data.covs.pickle    100%[===================>]   1.56G  23.0MB/s    in 72s     

2019-08-16 10:53:39 (22.1 MB/s) - ‘data.covs.pickle’ saved [1678972335/1678972335]



In [0]:
data_ext = pd.read_pickle('data.covs.pickle')

In [115]:
data_ext.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,rc_A,rc_B,rc_C,mu,alpha,homo,lumo,gap,r2,zpve,U0,U,H,G,Cv,freqs_min,freqs_max,freqs_mean,linear,mulliken_min,mulliken_max,mulliken_mean,mulliken_atom_0,mulliken_atom_1
0,5174511,dsgdb9nsd_033805,11,7,2JHC,NaN,3.54257,1.50643,1.34544,4.4029,76.7,-0.2457,-0.0269,0.2188,964.8507,0.126725,-363.615135,-363.608497,-363.607553,-363.645885,27.528,155.249,3252.0483,1324.421867,1.0,-0.342191,0.15667,-6.250000e-08,0.075457,-0.088328
1,5174510,dsgdb9nsd_033805,11,6,3JHC,NaN,3.54257,1.50643,1.34544,4.4029,76.7,-0.2457,-0.0269,0.2188,964.8507,0.126725,-363.615135,-363.608497,-363.607553,-363.645885,27.528,155.249,3252.0483,1324.421867,1.0,-0.342191,0.15667,-6.250000e-08,0.075457,-0.101114
2,5174509,dsgdb9nsd_033805,11,5,2JHC,NaN,3.54257,1.50643,1.34544,4.4029,76.7,-0.2457,-0.0269,0.2188,964.8507,0.126725,-363.615135,-363.608497,-363.607553,-363.645885,27.528,155.249,3252.0483,1324.421867,1.0,-0.342191,0.15667,-6.250000e-08,0.075457,-0.088328
3,5174508,dsgdb9nsd_033805,11,4,1JHC,NaN,3.54257,1.50643,1.34544,4.4029,76.7,-0.2457,-0.0269,0.2188,964.8507,0.126725,-363.615135,-363.608497,-363.607553,-363.645885,27.528,155.249,3252.0483,1324.421867,1.0,-0.342191,0.15667,-6.250000e-08,0.075457,-0.139609
4,5174507,dsgdb9nsd_033805,11,3,2JHC,NaN,3.54257,1.50643,1.34544,4.4029,76.7,-0.2457,-0.0269,0.2188,964.8507,0.126725,-363.615135,-363.608497,-363.607553,-363.645885,27.528,155.249,3252.0483,1324.421867,1.0,-0.342191,0.15667,-6.250000e-08,0.075457,-0.221362


In [0]:
data_ext.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,7.16369e+06,NaN,NaN,NaN,3.58184e+06,2.06798e+06,0,1.79092e+06,3.58184e+06,5.37277e+06,7.16369e+06
molecule_name,7163689,130775,dsgdb9nsd_042139,135,NaN,NaN,NaN,NaN,NaN,NaN,NaN
atom_index_0,7.16369e+06,NaN,NaN,NaN,13.356,3.26929,0,11,13,16,28
atom_index_1,7.16369e+06,NaN,NaN,NaN,5.88186,4.99337,0,2,5,8,28
type,7163689,8,3JHC,2322378,NaN,NaN,NaN,NaN,NaN,NaN,NaN
scalar_coupling_constant,4.65815e+06,NaN,NaN,NaN,15.9216,34.942,-36.2186,-0.254978,2.28113,7.39065,204.88
rc_A,7.16369e+06,NaN,NaN,NaN,4.00968,605.934,0,2.51725,3.00923,3.69287,619868
rc_B,7.16369e+06,NaN,NaN,NaN,1.38288,0.577625,0.33712,1.08126,1.36208,1.64268,437.904
rc_C,7.16369e+06,NaN,NaN,NaN,1.12456,0.433005,0.33118,0.91168,1.09291,1.29433,282.945
mu,7.16369e+06,NaN,NaN,NaN,2.4649,1.46241,0,1.4386,2.2212,3.3611,25.2022


In [116]:
data_ext.isna().describe(include='all')

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,rc_A,rc_B,rc_C,mu,alpha,homo,lumo,gap,r2,zpve,U0,U,H,G,Cv,freqs_min,freqs_max,freqs_mean,linear,mulliken_min,mulliken_max,mulliken_mean,mulliken_atom_0,mulliken_atom_1
count,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689
unique,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
freq,7163689,7163689,7163689,7163689,7163689,4658147,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7163689,7147479,7163689,7163689,7163689,7163689,7163689


In [0]:
data_ext['linear'].isna().value_counts()

False    7147479
True       16210
Name: linear, dtype: int64

In [0]:
data_ext['scalar_coupling_constant'].isna().value_counts()

False    4658147
True     2505542
Name: scalar_coupling_constant, dtype: int64

In [117]:
data_ext.drop('scalar_coupling_constant',axis=1,inplace=True)
data_ext.head()

,id,molecule_name,atom_index_0,atom_index_1,type,rc_A,rc_B,rc_C,mu,alpha,homo,lumo,gap,r2,zpve,U0,U,H,G,Cv,freqs_min,freqs_max,freqs_mean,linear,mulliken_min,mulliken_max,mulliken_mean,mulliken_atom_0,mulliken_atom_1
0,5174511,dsgdb9nsd_033805,11,7,2JHC,3.54257,1.50643,1.34544,4.4029,76.7,-0.2457,-0.0269,0.2188,964.8507,0.126725,-363.615135,-363.608497,-363.607553,-363.645885,27.528,155.249,3252.0483,1324.421867,1.0,-0.342191,0.15667,-6.250000e-08,0.075457,-0.088328
1,5174510,dsgdb9nsd_033805,11,6,3JHC,3.54257,1.50643,1.34544,4.4029,76.7,-0.2457,-0.0269,0.2188,964.8507,0.126725,-363.615135,-363.608497,-363.607553,-363.645885,27.528,155.249,3252.0483,1324.421867,1.0,-0.342191,0.15667,-6.250000e-08,0.075457,-0.101114
2,5174509,dsgdb9nsd_033805,11,5,2JHC,3.54257,1.50643,1.34544,4.4029,76.7,-0.2457,-0.0269,0.2188,964.8507,0.126725,-363.615135,-363.608497,-363.607553,-363.645885,27.528,155.249,3252.0483,1324.421867,1.0,-0.342191,0.15667,-6.250000e-08,0.075457,-0.088328
3,5174508,dsgdb9nsd_033805,11,4,1JHC,3.54257,1.50643,1.34544,4.4029,76.7,-0.2457,-0.0269,0.2188,964.8507,0.126725,-363.615135,-363.608497,-363.607553,-363.645885,27.528,155.249,3252.0483,1324.421867,1.0,-0.342191,0.15667,-6.250000e-08,0.075457,-0.139609
4,5174507,dsgdb9nsd_033805,11,3,2JHC,3.54257,1.50643,1.34544,4.4029,76.7,-0.2457,-0.0269,0.2188,964.8507,0.126725,-363.615135,-363.608497,-363.607553,-363.645885,27.528,155.249,3252.0483,1324.421867,1.0,-0.342191,0.15667,-6.250000e-08,0.075457,-0.221362


In [0]:
data_ext=data_ext.fillna(0)

In [0]:
data_ext.isna().describe(include='all')

,count,unique,top,freq
id,7163689,1,False,7163689
molecule_name,7163689,1,False,7163689
atom_index_0,7163689,1,False,7163689
atom_index_1,7163689,1,False,7163689
type,7163689,1,False,7163689
rc_A,7163689,1,False,7163689
rc_B,7163689,1,False,7163689
rc_C,7163689,1,False,7163689
mu,7163689,1,False,7163689
alpha,7163689,1,False,7163689


In [0]:
data_ext.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,7.16369e+06,NaN,NaN,NaN,3.58184e+06,2.06798e+06,0,1.79092e+06,3.58184e+06,5.37277e+06,7.16369e+06
molecule_name,7163689,130775,dsgdb9nsd_042139,135,NaN,NaN,NaN,NaN,NaN,NaN,NaN
atom_index_0,7.16369e+06,NaN,NaN,NaN,13.356,3.26929,0,11,13,16,28
atom_index_1,7.16369e+06,NaN,NaN,NaN,5.88186,4.99337,0,2,5,8,28
type,7163689,8,3JHC,2322378,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rc_A,7.16369e+06,NaN,NaN,NaN,4.00968,605.934,0,2.51725,3.00923,3.69287,619868
rc_B,7.16369e+06,NaN,NaN,NaN,1.38288,0.577625,0.33712,1.08126,1.36208,1.64268,437.904
rc_C,7.16369e+06,NaN,NaN,NaN,1.12456,0.433005,0.33118,0.91168,1.09291,1.29433,282.945
mu,7.16369e+06,NaN,NaN,NaN,2.4649,1.46241,0,1.4386,2.2212,3.3611,25.2022
alpha,7.16369e+06,NaN,NaN,NaN,77.382,7.64789,6.31,72.61,77.63,82.38,143.53


In [0]:
data_ext.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type', 'rc_A',
       'rc_B', 'rc_C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve',
       'U0', 'U', 'H', 'G', 'Cv', 'freqs_min', 'freqs_max', 'freqs_mean',
       'linear', 'mulliken_min', 'mulliken_max', 'mulliken_mean',
       'mulliken_atom_0', 'mulliken_atom_1'],
      dtype='object')

In [0]:
train.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'atom_0', 'x_0',
       'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'mulliken_charge_0',
       'mulliken_charge_1', 'type', 'scalar_coupling_constant', 'distance',
       'type_enc', 'atom_1_enc', 'atom_0_enc'],
      dtype='object')

In [0]:
train=pd.merge(train,data_ext,on=['id','molecule_name','atom_index_0','type','atom_index_1'],how='left')
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,atom_1_enc,atom_0_enc,rc_A,rc_B,rc_C,mu,alpha,homo,lumo,gap,r2,zpve,U0,U,H,G,Cv,freqs_min,freqs_max,freqs_mean,linear,mulliken_min,mulliken_max,mulliken_mean,mulliken_atom_0,mulliken_atom_1
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0,0,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133921,-0.535689
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3,1,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133921,0.133922
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3,1,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133921,0.133923
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3,1,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133921,0.133923
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0,0,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133922,-0.535689


In [0]:
train.shape

(4658147, 44)

# Feature Engineering (test)

## Mulliken Charge (Imputation)

In [33]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,2JHC
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,1JHC
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN,3JHH
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,1JHC
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,2JHC


In [34]:
avg_charge=pd.DataFrame(train.groupby('atom_1')['mulliken_charge_1'].mean()).reset_index()
avg_charge.head()

,atom_1,mulliken_charge_1
0,C,-0.064872
1,H,0.118187
2,N,-0.269914


In [35]:
test.drop('mulliken_charge_1',axis=1,inplace=True)
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,2JHC
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,1JHC
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,3JHH
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,1JHC
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,2JHC


In [36]:
test=pd.merge(test,avg_charge,on='atom_1',how='left')
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,2JHC,-0.064872
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,1JHC,-0.064872
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,3JHH,0.118187
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,1JHC,-0.064872
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,2JHC,-0.064872


In [37]:
#only H is present
test['mulliken_charge_0']=train['mulliken_charge_0'].mean()
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872


## Euclidian Distance

In [38]:
test['distance']=get_distance(test)
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1,distance
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187,3.323277
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178


## one-hot encode atom names
(Not neccessary)

In [0]:
#test=test.join(pd.get_dummies(test['atom_0'],prefix='atom_0'))
#test.head()

In [0]:
#test=test.join(pd.get_dummies(test['atom_1'],prefix='atom_1'))
#test.head()

## Label Encode type

In [39]:
test['type_enc']=le_type.transform(test['type'])
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187,3.323277,6
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2


## Label Encode Atoms

In [40]:
test['atom_0_enc']=le_atoms.transform(test['atom_0'])
test['atom_1_enc']=le_atoms.transform(test['atom_1'])
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,atom_0_enc,atom_1_enc
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,1,0
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,1,0
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187,3.323277,6,1,1
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,1,0
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,1,0


## Pairs

In [0]:
test['pairs']=(test['atom_0']+'_'+test['atom_1']).apply(lambda x:x.split('_')).apply(sorted).apply(lambda x:'_'.join(x)).apply(str.lower)

In [42]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,atom_0_enc,atom_1_enc,pairs
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,1,0,c_h
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,1,0,c_h
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187,3.323277,6,1,1,h_h
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,1,0,c_h
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,1,0,c_h


## Pairs Label Encode

In [43]:
pairs_dict

{'c_c': 0,
 'c_f': 1,
 'c_h': 2,
 'c_n': 3,
 'c_o': 4,
 'c_si': 5,
 'f_f': 6,
 'f_h': 7,
 'f_n': 8,
 'f_o': 9,
 'f_si': 10,
 'h_h': 11,
 'h_n': 12,
 'h_o': 13,
 'h_si': 14,
 'n_n': 15,
 'n_o': 16,
 'o_o': 17}

In [44]:
test['pairs'].unique()

array(['c_h', 'h_h', 'h_n'], dtype=object)

In [0]:
test['pairs_enc']=test['pairs'].map(pairs_dict)

In [46]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,atom_0_enc,atom_1_enc,pairs,pairs_enc
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,1,0,c_h,2
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,1,0,c_h,2
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187,3.323277,6,1,1,h_h,11
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,1,0,c_h,2
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,1,0,c_h,2


## Connectivity

In [0]:
x=test[['pairs_enc','distance']]

In [48]:
x.head()

,pairs_enc,distance
0,2,2.261178
1,2,1.062099
2,11,3.323277
3,2,1.062099
4,2,2.261178


In [49]:
x.shape

(2505542, 2)

In [50]:
test['bond']=model_nn.predict(x,verbose=1,batch_size=10000)

2505542/2505542 [==============================] - 2s 1us/sample


In [51]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,atom_0_enc,atom_1_enc,pairs,pairs_enc,bond
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,1,0,c_h,2,0.413226
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,1,0,c_h,2,0.917924
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187,3.323277,6,1,1,h_h,11,0.173375
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,1,0,c_h,2,0.917924
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,1,0,c_h,2,0.413226


In [52]:
test.shape

(2505542, 22)

In [53]:
test['bond'].isna().value_counts()

False    2505542
Name: bond, dtype: int64

In [0]:
test['bond_int']=test['bond'].apply(lambda x:int(x>0.5))

In [55]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,atom_0_enc,atom_1_enc,pairs,pairs_enc,bond,bond_int
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,1,0,c_h,2,0.413226,0
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,1,0,c_h,2,0.917924,1
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187,3.323277,6,1,1,h_h,11,0.173375,0
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,1,0,c_h,2,0.917924,1
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,1,0,c_h,2,0.413226,0


## Total Charge

In [0]:
test['total_charge']=test['mulliken_charge_0']+test['mulliken_charge_1']

In [0]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,...,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,...,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,...,0.12211,3JHH,0.118187,3.323277,6,h_h,11,0.173375,0,0.240297
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,...,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,...,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,...,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,...,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,...,0.12211,3JHH,0.118187,3.323277,6,h_h,11,0.173375,0,0.240297
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,...,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,...,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238


## Features from External Dataset

In [0]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,distance,type_enc,atom_0_enc,atom_1_enc
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,2JHC,2.261178,2,1,0
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,1JHC,1.062099,0,1,0
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN,3JHH,3.323277,6,1,1
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,1JHC,1.062099,0,1,0
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,2JHC,2.261178,2,1,0


In [0]:
test=pd.merge(test,data_ext,on=['id','molecule_name','atom_index_0','type','atom_index_1'],how='left')
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,distance,type_enc,atom_0_enc,atom_1_enc,scalar_coupling_constant,rc_A,rc_B,rc_C,mu,alpha,homo,lumo,gap,r2,zpve,U0,U,H,G,Cv,freqs_min,freqs_max,freqs_mean,linear,mulliken_min,mulliken_max,mulliken_mean,mulliken_atom_0,mulliken_atom_1
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,2JHC,2.261178,2,1,0,NaN,0.0,35.610036,35.610036,0.0,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,549.7648,3557.8599,1683.122114,0.0,-0.207019,0.207019,0.0,0.207019,-0.207019
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,1JHC,1.062099,0,1,0,NaN,0.0,35.610036,35.610036,0.0,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,549.7648,3557.8599,1683.122114,0.0,-0.207019,0.207019,0.0,0.207019,-0.207019
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN,3JHH,3.323277,6,1,1,NaN,0.0,35.610036,35.610036,0.0,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,549.7648,3557.8599,1683.122114,0.0,-0.207019,0.207019,0.0,0.207019,0.207019
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,1JHC,1.062099,0,1,0,NaN,0.0,35.610036,35.610036,0.0,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,549.7648,3557.8599,1683.122114,0.0,-0.207019,0.207019,0.0,0.207019,-0.207019
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,2JHC,2.261178,2,1,0,NaN,0.0,35.610036,35.610036,0.0,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,549.7648,3557.8599,1683.122114,0.0,-0.207019,0.207019,0.0,0.207019,-0.207019


## Dipole Moment

In [0]:
test['dp']=(-1*test['mulliken_charge_0']*test['distance']/2)+(test['mulliken_charge_1']*test['distance']/2)

In [0]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge,dp
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,...,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238,-0.211400
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,...,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238,-0.099297
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,...,3JHH,0.118187,3.323277,6,h_h,11,0.173375,0,0.240297,-0.006518
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,...,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238,-0.099297
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,...,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238,-0.211400


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge,dp
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,...,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238,-0.211400
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,...,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238,-0.099297
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,...,3JHH,0.118187,3.323277,6,h_h,11,0.173375,0,0.240297,-0.006518
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,...,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238,-0.099297
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,...,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238,-0.211400


# Normalize

In [56]:
train.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'atom_0', 'x_0',
       'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'mulliken_charge_0',
       'mulliken_charge_1', 'type', 'scalar_coupling_constant', 'distance',
       'type_enc', 'atom_1_enc', 'atom_0_enc', 'pairs', 'pairs_enc', 'bond',
       'bond_int'],
      dtype='object')

In [57]:
train.dtypes

id                            int64
molecule_name                object
atom_index_0                  int64
atom_index_1                  int64
atom_0                       object
x_0                         float64
y_0                         float64
z_0                         float64
atom_1                       object
x_1                         float64
y_1                         float64
z_1                         float64
mulliken_charge_0           float64
mulliken_charge_1           float64
type                         object
scalar_coupling_constant    float64
distance                    float64
type_enc                      int64
atom_1_enc                    int64
atom_0_enc                    int64
pairs                        object
pairs_enc                     int64
bond                        float32
bond_int                      int64
dtype: object

In [58]:
print(train.shape)
print(test.shape)

(4658147, 24)
(2505542, 23)


In [0]:
norm_cols=[ 'x_0','y_0','z_0','x_1','y_1','z_1','mulliken_charge_0','mulliken_charge_1','distance']

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
std_scaler=StandardScaler()
train[norm_cols]=std_scaler.fit_transform(train[norm_cols])

In [90]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,atom_1_enc,atom_0_enc,pairs,pairs_enc,bond,bond_int
0,0,dsgdb9nsd_000001,1,0,H,-0.058940,0.101017,-0.028861,C,-0.074536,0.838296,-0.055483,0.263942,-2.057703,1JHC,84.8076,-1.722482,0,0,1,c_h,2,0.919759,1
1,1,dsgdb9nsd_000001,1,2,H,-0.058940,0.101017,-0.028861,H,0.617919,1.052165,-0.061633,0.263942,0.729230,2JHH,-11.2570,-0.784301,3,1,1,h_h,11,0.648310,1
2,2,dsgdb9nsd_000001,1,3,H,-0.058940,0.101017,-0.028861,H,-0.431513,1.042984,-0.759868,0.263942,0.729234,2JHH,-11.2548,-0.784264,3,1,1,h_h,11,0.648205,1
3,3,dsgdb9nsd_000001,1,4,H,-0.058940,0.101017,-0.028861,H,-0.420021,1.037555,0.659852,0.263942,0.729234,2JHH,-11.2543,-0.784251,3,1,1,h_h,11,0.648170,1
4,4,dsgdb9nsd_000001,2,0,H,0.514726,0.793727,-0.029947,C,-0.074536,0.838296,-0.055483,0.263964,-2.057703,1JHC,84.8074,-1.722484,0,0,1,c_h,2,0.919760,1


In [91]:
train.shape

(4658147, 24)

In [93]:
train.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,4.65815e+06,NaN,NaN,NaN,2.32907e+06,1.34469e+06,0,1.16454e+06,2.32907e+06,3.49361e+06,4.65815e+06
molecule_name,4658147,85003,dsgdb9nsd_042139,135,NaN,NaN,NaN,NaN,NaN,NaN,NaN
atom_index_0,4.65815e+06,NaN,NaN,NaN,13.3569,3.26771,0,11,13,16,28
atom_index_1,4.65815e+06,NaN,NaN,NaN,5.88397,4.99394,0,2,5,8,28
atom_0,4658147,1,H,4658147,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x_0,4.65815e+06,NaN,NaN,NaN,-1.76683e-15,1,-5.30764,-0.601731,0.0160484,0.637471,5.27104
y_0,4.65815e+06,NaN,NaN,NaN,3.90423e-15,1,-4.37074,-0.761903,-0.0365845,0.930247,4.68229
z_0,4.65815e+06,NaN,NaN,NaN,5.81794e-15,1,-5.86885,-0.646426,-0.0269451,0.631489,4.85164
atom_1,4658147,3,C,3360469,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x_1,4.65815e+06,NaN,NaN,NaN,7.14353e-15,1,-6.30821,-0.601813,-0.0344512,0.588511,5.62609


In [0]:
test[norm_cols]=std_scaler.transform(test[norm_cols])

In [95]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,distance,type_enc,atom_1_enc,atom_0_enc,pairs,pairs_enc,bond,bond_int
0,4658147,dsgdb9nsd_000004,2,0,H,-1.004343,0.10386,0.609053,C,0.339301,0.223874,0.734382,-7.526702e-13,-0.098154,2JHC,-0.135391,2,0,1,c_h,2,0.413226,0
1,4658148,dsgdb9nsd_000004,2,1,H,-1.004343,0.10386,0.609053,C,-0.471207,0.223874,0.734382,-7.526702e-13,-0.098154,1JHC,-1.763005,0,0,1,c_h,2,0.917924,1
2,4658149,dsgdb9nsd_000004,2,3,H,-1.004343,0.10386,0.609053,H,1.057219,0.223874,0.734382,-7.526702e-13,0.663743,3JHH,1.306289,6,1,1,h_h,11,0.173375,0
3,4658150,dsgdb9nsd_000004,3,0,H,0.884018,0.10386,0.609053,C,0.339301,0.223874,0.734382,-7.526702e-13,-0.098154,1JHC,-1.763005,0,0,1,c_h,2,0.917924,1
4,4658151,dsgdb9nsd_000004,3,1,H,0.884018,0.10386,0.609053,C,-0.471207,0.223874,0.734382,-7.526702e-13,-0.098154,2JHC,-0.135391,2,0,1,c_h,2,0.413226,0


In [96]:
test.shape

(2505542, 23)

In [97]:
test.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,2.50554e+06,NaN,NaN,NaN,5.91092e+06,723288,4.65815e+06,5.28453e+06,5.91092e+06,6.5373e+06,7.16369e+06
molecule_name,2505542,45772,dsgdb9nsd_096326,131,NaN,NaN,NaN,NaN,NaN,NaN,NaN
atom_index_0,2.50554e+06,NaN,NaN,NaN,13.3545,3.27221,1,11,13,16,28
atom_index_1,2.50554e+06,NaN,NaN,NaN,5.87795,4.99231,0,2,5,8,28
atom_0,2505542,1,H,2505542,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x_0,2.50554e+06,NaN,NaN,NaN,0.00340874,0.995141,-4.8081,-0.597912,0.0189876,0.638736,5.14306
y_0,2.50554e+06,NaN,NaN,NaN,-0.00138942,0.995765,-4.57801,-0.759869,-0.0376564,0.927555,4.90262
z_0,2.50554e+06,NaN,NaN,NaN,0.00394271,1.00163,-5.03067,-0.644163,-0.0251377,0.634845,5.01601
atom_1,2505542,3,C,1805746,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x_1,2.50554e+06,NaN,NaN,NaN,0.00310345,0.993974,-5.71397,-0.590419,-0.0340159,0.588915,5.46947


# Drop & Rearrange Columns

In [98]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,atom_1_enc,atom_0_enc,pairs,pairs_enc,bond,bond_int
0,0,dsgdb9nsd_000001,1,0,H,-0.058940,0.101017,-0.028861,C,-0.074536,0.838296,-0.055483,0.263942,-2.057703,1JHC,84.8076,-1.722482,0,0,1,c_h,2,0.919759,1
1,1,dsgdb9nsd_000001,1,2,H,-0.058940,0.101017,-0.028861,H,0.617919,1.052165,-0.061633,0.263942,0.729230,2JHH,-11.2570,-0.784301,3,1,1,h_h,11,0.648310,1
2,2,dsgdb9nsd_000001,1,3,H,-0.058940,0.101017,-0.028861,H,-0.431513,1.042984,-0.759868,0.263942,0.729234,2JHH,-11.2548,-0.784264,3,1,1,h_h,11,0.648205,1
3,3,dsgdb9nsd_000001,1,4,H,-0.058940,0.101017,-0.028861,H,-0.420021,1.037555,0.659852,0.263942,0.729234,2JHH,-11.2543,-0.784251,3,1,1,h_h,11,0.648170,1
4,4,dsgdb9nsd_000001,2,0,H,0.514726,0.793727,-0.029947,C,-0.074536,0.838296,-0.055483,0.263964,-2.057703,1JHC,84.8074,-1.722484,0,0,1,c_h,2,0.919760,1


In [99]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,distance,type_enc,atom_1_enc,atom_0_enc,pairs,pairs_enc,bond,bond_int
0,4658147,dsgdb9nsd_000004,2,0,H,-1.004343,0.10386,0.609053,C,0.339301,0.223874,0.734382,-7.526702e-13,-0.098154,2JHC,-0.135391,2,0,1,c_h,2,0.413226,0
1,4658148,dsgdb9nsd_000004,2,1,H,-1.004343,0.10386,0.609053,C,-0.471207,0.223874,0.734382,-7.526702e-13,-0.098154,1JHC,-1.763005,0,0,1,c_h,2,0.917924,1
2,4658149,dsgdb9nsd_000004,2,3,H,-1.004343,0.10386,0.609053,H,1.057219,0.223874,0.734382,-7.526702e-13,0.663743,3JHH,1.306289,6,1,1,h_h,11,0.173375,0
3,4658150,dsgdb9nsd_000004,3,0,H,0.884018,0.10386,0.609053,C,0.339301,0.223874,0.734382,-7.526702e-13,-0.098154,1JHC,-1.763005,0,0,1,c_h,2,0.917924,1
4,4658151,dsgdb9nsd_000004,3,1,H,0.884018,0.10386,0.609053,C,-0.471207,0.223874,0.734382,-7.526702e-13,-0.098154,2JHC,-0.135391,2,0,1,c_h,2,0.413226,0


In [100]:
print('train: {}'.format(train.shape))
print('test: {}'.format(test.shape))

train: (4658147, 24)
test: (2505542, 23)


In [64]:
train.dtypes

id                            int64
molecule_name                object
atom_index_0                  int64
atom_index_1                  int64
atom_0                       object
x_0                         float64
y_0                         float64
z_0                         float64
atom_1                       object
x_1                         float64
y_1                         float64
z_1                         float64
mulliken_charge_0           float64
mulliken_charge_1           float64
type                         object
scalar_coupling_constant    float64
distance                    float64
type_enc                      int64
atom_1_enc                    int64
atom_0_enc                    int64
pairs                        object
pairs_enc                     int64
bond                        float32
bond_int                      int64
dtype: object

In [65]:
train.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'atom_0', 'x_0',
       'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'mulliken_charge_0',
       'mulliken_charge_1', 'type', 'scalar_coupling_constant', 'distance',
       'type_enc', 'atom_1_enc', 'atom_0_enc', 'pairs', 'pairs_enc', 'bond',
       'bond_int'],
      dtype='object')

In [68]:
train.corr()['scalar_coupling_constant']

id                         -0.006968
atom_index_0                0.018839
atom_index_1               -0.218989
x_0                        -0.006015
y_0                        -0.013119
z_0                         0.004807
x_1                         0.005132
y_1                         0.021189
z_1                        -0.007792
mulliken_charge_0          -0.025444
mulliken_charge_1          -0.360745
scalar_coupling_constant    1.000000
distance                   -0.668663
type_enc                   -0.641988
atom_1_enc                 -0.220403
atom_0_enc                       NaN
pairs_enc                  -0.258580
bond                        0.741879
bond_int                    0.659256
Name: scalar_coupling_constant, dtype: float64

In [0]:
drop_cols=['id', 'molecule_name', 'atom_0',  'atom_1', 'type','mulliken_charge_0', 'mulliken_charge_1' ]

In [0]:
#train.drop(drop_cols,axis=1,inplace=True)
#train.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,scalar_coupling_constant,distance,type_enc,atom_1_enc,atom_0_enc,rc_A,rc_B,rc_C,mu,alpha,homo,lumo,gap,r2,zpve,U0,U,H,G,Cv,freqs_min,freqs_max,freqs_mean,linear,mulliken_min,mulliken_max,mulliken_mean,mulliken_atom_0,mulliken_atom_1
0,1,0,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,84.8076,1.091953,0,0,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133921,-0.535689
1,1,2,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,-11.2570,1.783120,3,1,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133921,0.133922
2,1,3,0.002150,-0.006031,0.001976,-0.540815,1.447527,-0.876644,-11.2548,1.783147,3,1,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133921,0.133923
3,1,4,0.002150,-0.006031,0.001976,-0.523814,1.437933,0.906397,-11.2543,1.783157,3,1,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133921,0.133923
4,2,0,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,84.8074,1.091952,0,0,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133922,-0.535689


In [0]:
test.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'atom_0', 'x_0',
       'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'mulliken_charge_0',
       'mulliken_charge_1', 'type', 'distance', 'type_enc', 'atom_0_enc',
       'atom_1_enc'],
      dtype='object')

In [0]:
drop_cols=['id', 'molecule_name', 'atom_0',  'atom_1', 'type','mulliken_charge_0', 'mulliken_charge_1','scalar_coupling_constant' ]

In [0]:
#test.drop(drop_cols,axis=1,inplace=True)
#test.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,distance,type_enc,atom_0_enc,atom_1_enc,scalar_coupling_constant,rc_A,rc_B,rc_C,mu,alpha,homo,lumo,gap,r2,zpve,U0,U,H,G,Cv,freqs_min,freqs_max,freqs_mean,linear,mulliken_min,mulliken_max,mulliken_mean,mulliken_atom_0,mulliken_atom_1
0,2,0,-1.661639,0.0,1.0,0.599539,0.0,1.0,2.261178,2,1,0,NaN,0.0,35.610036,35.610036,0.0,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,549.7648,3557.8599,1683.122114,0.0,-0.207019,0.207019,0.0,0.207019,-0.207019
1,2,1,-1.661639,0.0,1.0,-0.599539,0.0,1.0,1.062099,0,1,0,NaN,0.0,35.610036,35.610036,0.0,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,549.7648,3557.8599,1683.122114,0.0,-0.207019,0.207019,0.0,0.207019,-0.207019
2,2,3,-1.661639,0.0,1.0,1.661639,0.0,1.0,3.323277,6,1,1,NaN,0.0,35.610036,35.610036,0.0,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,549.7648,3557.8599,1683.122114,0.0,-0.207019,0.207019,0.0,0.207019,0.207019
3,3,0,1.661639,0.0,1.0,0.599539,0.0,1.0,1.062099,0,1,0,NaN,0.0,35.610036,35.610036,0.0,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,549.7648,3557.8599,1683.122114,0.0,-0.207019,0.207019,0.0,0.207019,-0.207019
4,3,1,1.661639,0.0,1.0,-0.599539,0.0,1.0,2.261178,2,1,0,NaN,0.0,35.610036,35.610036,0.0,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,549.7648,3557.8599,1683.122114,0.0,-0.207019,0.207019,0.0,0.207019,-0.207019


In [69]:
train.isna().describe()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,atom_1_enc,atom_0_enc,pairs,pairs_enc,bond,bond_int
count,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147
unique,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
freq,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147


In [70]:
test.isna().describe()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,atom_0_enc,atom_1_enc,pairs,pairs_enc,bond,bond_int
count,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542
unique,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
freq,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542


In [101]:
train.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'atom_0', 'x_0',
       'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'mulliken_charge_0',
       'mulliken_charge_1', 'type', 'scalar_coupling_constant', 'distance',
       'type_enc', 'atom_1_enc', 'atom_0_enc', 'pairs', 'pairs_enc', 'bond',
       'bond_int'],
      dtype='object')

In [0]:
train=train[['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'atom_0', 'x_0',
       'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'mulliken_charge_0',
       'mulliken_charge_1', 'type', 'scalar_coupling_constant', 'distance',
       'type_enc', 'atom_1_enc', 'atom_0_enc', 'pairs', 'pairs_enc', 'bond',
       'bond_int']]
train.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,distance,type_enc,atom_1_enc,atom_0_enc,rc_A,rc_B,rc_C,mu,alpha,homo,lumo,gap,r2,zpve,U0,U,H,G,Cv,freqs_min,freqs_max,freqs_mean,linear,mulliken_min,mulliken_max,mulliken_mean,mulliken_atom_0,mulliken_atom_1,scalar_coupling_constant
0,1,0,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,1.091953,0,0,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133921,-0.535689,84.8076
1,1,2,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,1.783120,3,1,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133921,0.133922,-11.2570
2,1,3,0.002150,-0.006031,0.001976,-0.540815,1.447527,-0.876644,1.783147,3,1,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133921,0.133923,-11.2548
3,1,4,0.002150,-0.006031,0.001976,-0.523814,1.437933,0.906397,1.783157,3,1,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133921,0.133923,-11.2543
4,2,0,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,1.091952,0,0,1,157.7118,157.70997,157.70699,0.0,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.47893,-40.476062,-40.475117,-40.498597,6.469,1341.307,3151.7078,2182.525478,1.0,-0.535689,0.133923,0.0,0.133922,-0.535689,84.8074


In [102]:
test.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'atom_0', 'x_0',
       'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'mulliken_charge_0',
       'mulliken_charge_1', 'type', 'distance', 'type_enc', 'atom_1_enc',
       'atom_0_enc', 'pairs', 'pairs_enc', 'bond', 'bond_int'],
      dtype='object')

In [72]:
test=test[['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'atom_0', 'x_0',
       'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'mulliken_charge_0',
       'mulliken_charge_1', 'type','distance',
       'type_enc', 'atom_1_enc', 'atom_0_enc', 'pairs', 'pairs_enc', 'bond',
       'bond_int']]
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,distance,type_enc,atom_1_enc,atom_0_enc,pairs,pairs_enc,bond,bond_int
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,-0.064872,2JHC,2.261178,2,0,1,c_h,2,0.413226,0
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,-0.064872,1JHC,1.062099,0,0,1,c_h,2,0.917924,1
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,0.118187,3JHH,3.323277,6,1,1,h_h,11,0.173375,0
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,-0.064872,1JHC,1.062099,0,0,1,c_h,2,0.917924,1
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,-0.064872,2JHC,2.261178,2,0,1,c_h,2,0.413226,0


# Serialize Data

In [73]:
print(train.shape)
print(test.shape)

(4658147, 24)
(2505542, 23)


In [0]:
train.to_csv('train_feat_eng_v2_orig_my_std.csv',index=False)

In [0]:
test.to_csv('test_feat_eng_v2_orig_my_std.csv',index=False)

In [105]:
!ls

connectivity_model_nn_03.h5	       structures.csv.zip
dipole_moments.csv.zip		       structures.zip
kaggle.json			       test.csv.zip
magnetic_shielding_tensors.csv.zip     test_etl.csv
mulliken_charges.csv.zip	       test_feat_eng_v2_orig_my_not_std.csv
pairs_dict.pkl			       test_feat_eng_v2_orig_my_std.csv
potential_energy.csv.zip	       train.csv.zip
sample_data			       train_etl.csv
sample_submission.csv.zip	       train_feat_eng_v2_orig_my_not_std.csv
scalar_coupling_contributions.csv.zip  train_feat_eng_v2_orig_my_std.csv


In [82]:
df=pd.read_csv('train_feat_eng_v2_orig_my_not_std.csv')
df.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,atom_1_enc,atom_0_enc,pairs,pairs_enc,bond,bond_int
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0,0,1,c_h,2,0.919759,1
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3,1,1,h_h,11,0.648310,1
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3,1,1,h_h,11,0.648205,1
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3,1,1,h_h,11,0.648170,1
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0,0,1,c_h,2,0.919759,1


In [83]:
df.shape

(4658147, 24)

In [108]:
df=pd.read_csv('test_feat_eng_v2_orig_my_std.csv')
df.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,distance,type_enc,atom_1_enc,atom_0_enc,pairs,pairs_enc,bond,bond_int
0,4658147,dsgdb9nsd_000004,2,0,H,-1.004343,0.10386,0.609053,C,0.339301,0.223874,0.734382,-7.526702e-13,-0.098154,2JHC,-0.135391,2,0,1,c_h,2,0.413226,0
1,4658148,dsgdb9nsd_000004,2,1,H,-1.004343,0.10386,0.609053,C,-0.471207,0.223874,0.734382,-7.526702e-13,-0.098154,1JHC,-1.763005,0,0,1,c_h,2,0.917924,1
2,4658149,dsgdb9nsd_000004,2,3,H,-1.004343,0.10386,0.609053,H,1.057219,0.223874,0.734382,-7.526702e-13,0.663743,3JHH,1.306289,6,1,1,h_h,11,0.173375,0
3,4658150,dsgdb9nsd_000004,3,0,H,0.884018,0.10386,0.609053,C,0.339301,0.223874,0.734382,-7.526702e-13,-0.098154,1JHC,-1.763005,0,0,1,c_h,2,0.917924,1
4,4658151,dsgdb9nsd_000004,3,1,H,0.884018,0.10386,0.609053,C,-0.471207,0.223874,0.734382,-7.526702e-13,-0.098154,2JHC,-0.135391,2,0,1,c_h,2,0.413226,0


In [109]:
df.shape

(2505542, 23)

# Download Files

In [87]:
from google.colab import files
files.download('train_feat_eng_v2_orig_my_not_std.csv') 
files.download('test_feat_eng_v2_orig_my_not_std.csv') 

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 36876, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

## Upload files to IBM Object Store

In [0]:
# The code was removed by Watson Studio for sharing.

In [0]:
from ibm_botocore.client import Config
import ibm_boto3
def upload_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Uploaded')

In [0]:
upload_file_cos(credentials_2,'train_feat_eng_v2_1.csv','train_feat_eng_v2_1.csv')
upload_file_cos(credentials_2,'train_feat_eng_v2_2.csv','train_feat_eng_v2_2.csv')

File Uploaded
File Uploaded


In [0]:
upload_file_cos(credentials_2,'test_feat_eng_v2.csv','test_feat_eng_v2.csv')

File Uploaded
